In [ ]:
import warnings
warnings.filterwarnings('ignore')

### Creating Data

In [ ]:
import numpy as np

In [ ]:
np.random.seed(42)

Creating input or Independent variable

In [ ]:
#Radians value for angles between  60 and 300
x = np.array([i*np.pi/180 for i in range(60, 300,4)])

In [ ]:
x.shape

Create Target Variable - Sine value with some noise

In [ ]:
y = np.sin(x) + np.random.normal(0.0, 0.15, x.shape[0])

Build a Dataframe

In [ ]:
import pandas as pd

In [ ]:
train_df = pd.DataFrame(np.column_stack([x,y]), columns=['x','y'])

In [ ]:
train_df.shape

### Visualize the data

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(train_df['x'],train_df['y'],'.')
plt.title('x vs sin(x)')
plt.show()

### Building a Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
model = LinearRegression(normalize=True)
model.fit(train_df[['x']], train_df[['y']])

Visualize Prediction

In [ ]:
#Actual data
plt.plot(train_df['x'], train_df['y'], '.')

#Prediction
plt.plot(train_df['x'], model.predict(train_df[['x']]))
plt.title('Actual vs Prediction')
plt.show()

### Add More Complex features
Build Polynomial features

In [ ]:
for i in range(2,16):
    
    train_df['x_' + str(i)] = train_df['x'] ** i

In [ ]:
train_df.head()

### Function to Build model with Polynomials

Build a Dataframe to store Results

In [ ]:
columns = ['Loss'] + ['W_%d'%i for i in range(1,16)]
idx = ['power_%d'%i for i in range(1,16)]
results_df = pd.DataFrame(index=idx, columns=columns)

In [ ]:
results_df

In [ ]:
plt.subplots(nrows=5,ncols=3, figsize=(15,15))

for i in range(1, 16):
    
    #Select Features to us
    features = ['x']
    
    for j in range(2,i+1):
        
        features.append('x_' + str(j))
           
    #Build Model
    model = LinearRegression(normalize=True)
    model.fit(train_df[features], train_df[['y']])
    
    y_pred = model.predict(train_df[features])
    
    #Plot the output
    plt.subplot(5,3,i)
    plt.plot(train_df['x'], train_df['y'], '.')
    plt.plot(train_df['x'], y_pred)
    plt.title('Actual vs Prediction - Power ' + str(i))
    
    #Calculate metrices
    #Loss or Error
    results_df.loc['power_' + str(i), 'Loss'] = np.sum(np.square(y_pred - np.array(train_df[['y']])))
    
    #Get Weights values
    for j in range(1, i+1):
        
        results_df.loc['power_%d'%i, 'W_%d'%j] = model.coef_[0][j-1]    
    
plt.show()

In [ ]:
results_df

### Build L2 Regression

In [ ]:
from sklearn.linear_model import Ridge

In [ ]:
lambda_l2 = [1e-10, 1e-8, 1e-4, 1e-3,1e-2, 1, 5, 10, 20]

In [ ]:
columns = ['Lambda','Loss'] + ['W_%d'%i for i in range(1,16)]
idx = ['lambda_%d'%i for i in range(1,len(lambda_l2)+1)]
L2_df = pd.DataFrame(index=idx, columns=columns)

In [ ]:
plt.subplots(nrows=3,ncols=3, figsize=(15,15))

for i in range(len(lambda_l2)):
    
    #Select all features
    features = ['x']
    
    for j in range(2,16):
        
        features.append('x_' + str(j))
           
    #Build Model
    model = Ridge(alpha=lambda_l2[i], normalize=True)
    model.fit(train_df[features], train_df[['y']])
    
    y_pred = model.predict(train_df[features])
    
    #Plot the output
    plt.subplot(3,3,i+1)
    plt.plot(train_df['x'], train_df['y'], '.')
    plt.plot(train_df['x'], y_pred)
    plt.title('Actual vs Prediction - Lambda ' + str(lambda_l2[i]))
    
    #Calculate metrices
    #Loss or Error
    L2_df.loc['lambda_' + str(i+1), 'Lambda'] = lambda_l2[i] 
    L2_df.loc['lambda_' + str(i+1), 'Loss'] = np.sum(np.square(y_pred - np.array(train_df[['y']])))
    
    #Get Weights values
    for j in range(1, 16):
        
        L2_df.loc['lambda_' + str(i+1), 'W_' + str(j)] = model.coef_[0][j-1]    
    
plt.show()

In [ ]:
L2_df

### L1 Regression (Lasso Regression)

In [ ]:
from sklearn.linear_model import Lasso

In [ ]:
lambda_l1 = [1e-10, 1e-8, 1e-5,1e-4, 1e-3,1e-2, 1, 5, 10]

In [ ]:
columns = ['Lambda','Loss'] + ['W_%d'%i for i in range(1,16)]
idx = ['lambda_%d'%i for i in range(1,len(lambda_l1)+1)]
L1_df = pd.DataFrame(index=idx, columns=columns)

In [ ]:
plt.subplots(nrows=3,ncols=3, figsize=(15,15))

for i in range(len(lambda_l1)):
    
    #Select all features
    features = ['x']
    
    for j in range(2,16):
        
        features.append('x_' + str(j))
           
    #Build Model
    model = Lasso(alpha=lambda_l1[i], normalize=True, max_iter=1e5)
    model.fit(train_df[features], train_df[['y']])
    
    y_pred = model.predict(train_df[features])
    
    #Plot the output
    plt.subplot(3,3,i+1)
    plt.plot(train_df['x'], train_df['y'], '.')
    plt.plot(train_df['x'], y_pred)
    plt.title('Actual vs Prediction - Lambda ' + str(lambda_l1[i]))
    
    #Calculate metrices
    #Loss or Error
    L1_df.loc['lambda_' + str(i+1), 'Lambda'] = lambda_l1[i] 
    L1_df.loc['lambda_' + str(i+1), 'Loss'] = np.sum(np.square(y_pred - np.array(train_df[['y']])))
    
    #Get Weights values
    for j in range(1, 16):
        
        L1_df.loc['lambda_' + str(i+1), 'W_' + str(j)] = model.coef_[j-1]    
    
plt.show()

In [ ]:
L1_df